In [31]:
import os
import pandas as pd
from openai import OpenAI
import requests
import yfinance as yf
from sklearn.linear_model import LinearRegression
import numpy as np

In [32]:
api_key = os.getenv("OPENAI_API_KEY")
news_api_key = os.getenv("NEWS_API_KEY")

# Check if the OpenAI API key is loaded
if api_key:
    print("OpenAI API key loaded successfully")
else:
    print("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable.")

# Check if the News API key is loaded
if news_api_key:
    print("News API key loaded successfully")
else:
    print("News API key not found. Please set the NEWS_API_KEY environment variable.")

OpenAI API key loaded successfully
News API key loaded successfully


In [33]:
def get_company_news(symbol, api_key=news_api_key):
    url = f'https://newsapi.org/v2/everything?q={symbol}&apiKey={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        return None
    
def analyze_sentiment(article_text, client):
    prompt = f"""
                Analyze the sentiment of the following financial news about the company. Consider the overall tone, keywords, and context to determine if it is positive, negative, or neutral. Respond with only one word: 'positive', 'negative', or 'neutral'.
                News content: {article_text}
            """
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.0
    )
    sentiment = response.choices[0].message.content.strip()
    return sentiment

def get_historical_data(symbol, start_date="2023-01-01", end_date="2024-01-01"):
    try:
        stock = yf.Ticker(symbol)
        data = stock.history(start=start_date, end=end_date)
        return data['Close']
    except Exception as e:
        print(f"Error fetching data for {symbol}: {e}")
        return None

def predict_stock_price(data):
    if data is None or len(data) < 2:
        return None  # Not enough data for prediction

    # Create an array representing the days (e.g., 0, 1, 2, ...)
    X = np.array(range(len(data))).reshape(-1, 1)
    y = data.values

    model = LinearRegression()
    model.fit(X, y)

    # Predict the next day's price
    next_day = np.array([[len(data)]])
    prediction = model.predict(next_day)

    return round(prediction[0], 2)


In [34]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

In [35]:
n100 = pd.read_csv("/Users/yuliaanashkina/Downloads/nasdaq100.csv")
n100_p = pd.read_csv("/Users/yuliaanashkina/Downloads/nasdaq100_price_change.csv")
nasdaq100 = n100.merge(n100_p[["symbol", "ytd"]], on="symbol", how="inner")
nasdaq100.head()

,symbol,name,headQuarter,dateFirstAdded,cik,founded,ytd
0,AAPL,Apple Inc.,"Cupertino, CA",NaN,320193,1976-04-01,42.99992
1,ABNB,Airbnb,"San Francisco, CA",NaN,1559720,2008-08-01,68.66902
2,ADBE,Adobe Inc.,"San Jose, CA",NaN,796343,1982-12-01,57.22723
3,ADI,Analog Devices,"Wilmington, MA",NaN,6281,1965-01-01,17.02062
4,ADP,ADP,"Roseland, NJ",NaN,8670,1949-01-01,5.53732


In [36]:
# Loop through the NASDAQ companies
for company in nasdaq100["symbol"]:
    # Create a prompt to enrich nasdaq100 using OpenAI
    prompt = f'''Classify company {company} into one of the following sectors. Answer only with the sector name: Technology, Consumer Cyclical, Industrials, Utilities, Healthcare, Communication, Energy, Consumer Defensive, Real Estate, Financial.
'''
    # Create a request to the completions endpoint
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{ "role": "user", "content": prompt}],
        temperature=0.0,
    )
    # Store the output as a variable called sector
    sector = response.choices[0].message.content
    
    # Add the sector for the corresponding company
    nasdaq100.loc[nasdaq100["symbol"] == company, "Sector"] = sector
    
    

In [40]:
for company in nasdaq100["symbol"]:
    # Fetch news and analyze sentiment
    if news_api_key:
        news_data = get_company_news(company, api_key=news_api_key)
        if news_data and 'articles' in news_data:
            article_texts = " ".join([
                (article.get('title') or '') + " " + (article.get('description') or '')
                for article in news_data['articles']
            ])
            sentiment = analyze_sentiment(article_texts, client)
        
        if len(article_texts) < 50:
            sentiment = 'neutral'
            print(f"Insufficient news content for {company}. Defaulting to 'neutral'.")
        else:
            sentiment = analyze_sentiment(article_texts, client)
            print(f"Sentiment for {company}: {sentiment}")
    else:
        sentiment = 'neutral'
        print(f"No news articles found for {company}. Defaulting to 'neutral'.")


    # Fetch historical data and predict price
    historical_data = get_historical_data(company)
    predicted_price = predict_stock_price(historical_data)

    # Update the DataFrame with sentiment and predicted price
    nasdaq100.loc[nasdaq100["symbol"] == company, "Sentiment"] = sentiment
    nasdaq100.loc[nasdaq100["symbol"] == company, "Predicted Price"] = predicted_price

    print(f"Company: {company}, Sentiment: {sentiment}, Predicted Price: {predicted_price}")


Sentiment for AAPL: negative
Company: AAPL, Sentiment: negative, Predicted Price: 196.11
Sentiment for ABNB: negative
Company: ABNB, Sentiment: negative, Predicted Price: 138.7
Sentiment for ADBE: negative
Company: ADBE, Sentiment: negative, Predicted Price: 622.28
Sentiment for ADI: neutral
Company: ADI, Sentiment: neutral, Predicted Price: 179.24
Sentiment for ADP: neutral
Company: ADP, Sentiment: neutral, Predicted Price: 233.02
Sentiment for ADSK: negative
Company: ADSK, Sentiment: negative, Predicted Price: 217.45
Sentiment for AEP: neutral
Company: AEP, Sentiment: neutral, Predicted Price: 72.05
Sentiment for ALGN: neutral
Company: ALGN, Sentiment: neutral, Predicted Price: 271.59
Sentiment for AMAT: negative
Company: AMAT, Sentiment: negative, Predicted Price: 155.4
Sentiment for AMD: neutral
Company: AMD, Sentiment: neutral, Predicted Price: 127.51
Sentiment for AMGN: neutral
Company: AMGN, Sentiment: neutral, Predicted Price: 260.43
Sentiment for AMZN: negative
Company: AMZN, 

$ATVI: possibly delisted; no timezone found


Company: ATVI, Sentiment: neutral, Predicted Price: None
Sentiment for AVGO: negative
Company: AVGO, Sentiment: negative, Predicted Price: 101.26
Sentiment for AZN: neutral
Company: AZN, Sentiment: neutral, Predicted Price: 64.94
Sentiment for BIIB: neutral
Company: BIIB, Sentiment: neutral, Predicted Price: 246.63
Sentiment for BKNG: negative
Company: BKNG, Sentiment: negative, Predicted Price: 3282.18
Sentiment for BKR: neutral
Company: BKR, Sentiment: neutral, Predicted Price: 34.72
Sentiment for CDNS: negative
Company: CDNS, Sentiment: negative, Predicted Price: 270.23
Sentiment for CEG: negative
Company: CEG, Sentiment: negative, Predicted Price: 120.67
Sentiment for CHTR: negative
Company: CHTR, Sentiment: negative, Predicted Price: 416.58
Sentiment for CMCSA: negative
Company: CMCSA, Sentiment: negative, Predicted Price: 43.95
Sentiment for COST: negative
Company: COST, Sentiment: negative, Predicted Price: 591.25
Sentiment for CPRT: negative
Company: CPRT, Sentiment: negative, 

$SGEN: possibly delisted; no timezone found


Company: SGEN, Sentiment: negative, Predicted Price: None
Sentiment for SIRI: neutral
Company: SIRI, Sentiment: neutral, Predicted Price: 44.29
Sentiment for SNPS: neutral
Company: SNPS, Sentiment: neutral, Predicted Price: 533.38
Sentiment for TEAM: neutral
Company: TEAM, Sentiment: neutral, Predicted Price: 206.8
Sentiment for TMUS: neutral
Company: TMUS, Sentiment: neutral, Predicted Price: 141.81
Sentiment for TSLA: neutral
Company: TSLA, Sentiment: neutral, Predicted Price: 268.79
Sentiment for TXN: neutral
Company: TXN, Sentiment: neutral, Predicted Price: 151.97
Sentiment for VRSK: negative
Company: VRSK, Sentiment: negative, Predicted Price: 252.64
Sentiment for VRTX: neutral
Company: VRTX, Sentiment: neutral, Predicted Price: 382.45
Sentiment for WBA: neutral
Company: WBA, Sentiment: neutral, Predicted Price: 18.68
Sentiment for WBD: neutral
Company: WBD, Sentiment: neutral, Predicted Price: 10.57
Sentiment for WDAY: neutral
Company: WDAY, Sentiment: neutral, Predicted Price: 

In [41]:
# Prompt to get stock recommendations
prompt = f'''Provide summary information about Nasdaq-100 stock performance year to date (YTD), recommending the three best sectors and three or more companies per sector. 
            Consider the sentiment (positive, negative, neutral) from news articles and the predicted stock prices.
            Company data: {nasdaq100} 
'''

# Get the model response
response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{ "role": "user", "content": prompt}],
        temperature=0.0,
    )

# Store the output as a variable and print the recommendations
stock_recommendations = response.choices[0].message.content
print(stock_recommendations)

The Nasdaq-100 stock performance year to date (YTD) shows a mix of positive, negative, and neutral sentiment across different sectors. The three best sectors based on YTD performance are Technology, Real Estate, and Financial.

1. Technology Sector:
- Apple Inc. (AAPL) - Predicted Price: $196.11
- Adobe Inc. (ADBE) - Predicted Price: $622.28
- Workday, Inc. (WDAY) - Predicted Price: $256.10

2. Real Estate Sector:
- Airbnb (ABNB) - Predicted Price: $138.70

3. Financial Sector:
- Warner Bros. Discovery (WBD) - Predicted Price: $10.57

It is important to note that the sentiment for some companies in the Technology sector is negative, while others have a neutral sentiment. Investors should consider this sentiment along with the predicted stock prices when making investment decisions.
